In [ ]:
# 🚀 PRODUCTION PROMOTION - NEW WORKFLOW (CONFIG-DRIVEN)
import mlflow
from mlflow.tracking import MlflowClient
import time
import yaml
import sys
import traceback
import requests
from typing import Optional, Dict, Tuple
from datetime import datetime
from pyspark.sql import SparkSession

print("=" * 80)
print("🚀 PRODUCTION PROMOTION (NEW WORKFLOW)")
print("=" * 80)

# ✅ LOAD PIPELINE CONFIGURATION

print("\n📋 Step 1: Loading configuration from pipeline_config.yml...")

try:
    import os

    config_path = "/Workspace/Repos/vipultak7171@gmail.com/ml-credit-risk/dev_env/pipeline_config.yml"

    if not os.path.exists(config_path):
        config_path = "/Workspace/ml-credit-risk/dev_env/pipeline_config.yml"
    
    with open(config_path, "r") as f:
        pipeline_cfg = yaml.safe_load(f)

    print(f"✅ Configuration loaded from: {config_path}")

except FileNotFoundError:
    print("❌ ERROR: pipeline_config.yml not found!")
    sys.exit(1)
except Exception as e:
    print(f"❌ ERROR loading configuration: {e}")
    traceback.print_exc()
    sys.exit(1)



# ================== CONFIG ==================

class Config:
    def __init__(self):
        MODEL_TYPE = pipeline_cfg["model"]["type"]
        UC_CATALOG = pipeline_cfg["model"]["catalog"]
        UC_SCHEMA = pipeline_cfg["model"]["schema"]
        BASE_NAME = pipeline_cfg["model"]["base_name"]
        
        self.MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.{BASE_NAME}_{MODEL_TYPE}"
        self.MODEL_TYPE = MODEL_TYPE
        
        self.STAGING_ALIAS = pipeline_cfg["aliases"]["staging"]
        self.PRODUCTION_ALIAS = pipeline_cfg["aliases"]["production"]
        
        self.PRIMARY_METRIC = pipeline_cfg["metrics"]["classification"]["primary_metric"]
        self.DIRECTION = pipeline_cfg["metrics"]["classification"]["direction"]
        
        self.UAT_ENABLED = pipeline_cfg["uat"]["enabled"]
        self.UAT_RESULTS_TABLE = pipeline_cfg["tables"]["uat_results"]

        self.SLACK_ENABLED = pipeline_cfg["notifications"]["enabled"]
        self.SLACK_WEBHOOK_URL = self._get_slack_webhook()
        
        self.TOLERANCE = 1e-6

        print(f"\n📊 Config Loaded → Model: {self.MODEL_NAME}, Slack: {self.SLACK_WEBHOOK_URL is not None}")

    def _get_slack_webhook(self) -> Optional[str]:
        if not self.SLACK_ENABLED:
            return None
        try:
            scopes = ["shared-scope", "dev-scope", "prod-scope"]
            for scope in scopes:
                try:
                    webhook = dbutils.secrets.get(scope, "SLACK_WEBHOOK_URL")
                    if webhook.strip():
                        return webhook
                except:
                    pass
            return None
        except:
            return None


config = Config()

print("=" * 80)



# ================== SLACK HANDLER ==================

class SlackNotifier:

    def __init__(self, webhook_url):
        self.webhook_url = webhook_url
        self.enabled = bool(webhook_url)

    def send(self, message, level="info", extra=None):
        """Safe Slack sender — never breaks pipeline."""
        
        if not self.enabled:
            print(f"📢 (Slack Disabled) {message}")
            return False

        emoji = {
            "info": "ℹ️",
            "success": "✅",
            "warning": "⚠️",
            "error": "❌",
            "rocket": "🚀"
        }.get(level, "ℹ️")

        formatted_message = f"{emoji} *{message}*"

        if extra:
            formatted_message += "\n"
            for k, v in extra.items():
                formatted_message += f"\n• *{k}:* {v}"

        try:
            response = requests.post(self.webhook_url, json={"text": formatted_message}, timeout=5)

            if response.status_code == 200:
                print("📨 Slack message sent successfully")
                return True
            
            print(f"⚠ Slack HTTP Error → {response.status_code}")
            return False

        except Exception as e:
            print(f"⚠ Slack send failed but safely ignored → {e}")
            return False


    def success(self, model_name: str, version: int, metrics: Dict):
        details = {
            "Model": model_name,
            "Version": f"v{version}",
            "Status": "🟢 LIVE in Production"
        }

        for metric_name, value in metrics.items():
            if value is not None:
                details[metric_name] = f"{value:.4f}" if isinstance(value, float) else str(value)

        self.send("🚀 Production Deployment Successful", level="rocket", extra=details)


    def blocked(self, reason: str):
        self.send("🚫 Production Promotion Blocked", level="warning", extra={"Reason": reason})


slack = SlackNotifier(config.SLACK_WEBHOOK_URL)



# ================== MLFLOW INIT ==================

print("\n🔧 Initializing MLflow...")

try:
    spark = SparkSession.builder.appName("ProductionPromotion").getOrCreate()
    mlflow.set_tracking_uri("databricks")
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()
    print("✅ MLflow Running")
except Exception as e:
    print(f"❌ Pipeline Init Failed: {e}")
    sys.exit(1)



# ================== HELPER FUNCTIONS ==================

def get_metric(run_id):
    try:
        return client.get_run(run_id).data.metrics.get(config.PRIMARY_METRIC)
    except:
        return None



# ================== STAGING MODEL ==================

def get_staging():
    print("\n📍 Fetching staging model...")
    try:
        mv = client.get_model_version_by_alias(config.MODEL_NAME, config.STAGING_ALIAS)
        return {"version": int(mv.version), "run": mv.run_id, "metric": get_metric(mv.run_id)}
    except:
        return None



# ================== UAT VALIDATION ==================

def check_uat(version):
    print("\n📍 Validating UAT results...")
    df = spark.table(config.UAT_RESULTS_TABLE).toPandas()

    res = df[df["model_version"].astype(str) == str(version)]
    if res.empty:
        return False, None

    latest = res.sort_values("timestamp").iloc[-1]
    return latest["uat_status"] == "PASSED", {
        "accuracy": latest["accuracy"],
        "precision": latest["precision"],
        "recall": latest["recall"],
        "f1": latest["f1"],
        "roc_auc": latest["roc_auc"]
    }



# ================== PROMOTE ==================

def promote(model, uat):
    v = model["version"]

    print(f"\n🚀 Setting @{config.PRODUCTION_ALIAS} → v{v}")

    client.set_registered_model_alias(config.MODEL_NAME, config.PRODUCTION_ALIAS, v)

    print("\n🎉 MODEL LIVE IN PRODUCTION!")
    slack.success(config.MODEL_NAME, v, uat)



# ================== MAIN ==================

def main():
    slack.send("🔥 Promotion Pipeline Started")

    model = get_staging()
    if not model:
        slack.blocked("No staging model found.")
        sys.exit(1)

    ok, uat = check_uat(model["version"])
    if not ok:
        slack.blocked("UAT Failed. Cannot promote.")
        sys.exit(1)

    promote(model, uat)

    print("\n✨ Workflow Complete!")


if __name__ == "__main__":
    main()
